# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
import pandas as pd
import seaborn as sns

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

In [ ]:
df = pd.read_csv('/datasets/autos.csv')
display(df.info())
display(df.describe())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [ ]:
df.columns = df.columns.str.lower()
display(df.columns)

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'kilometer', 'registrationmonth', 'fueltype', 'brand',
       'repaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

### Удалим ненужные столбцы

In [ ]:
dropped_columns = ['datecrawled', 'registrationmonth', 'datecreated', 'numberofpictures', 'postalcode', 'lastseen']
df = df.drop(dropped_columns, axis=1)

### Обработка пропусков

In [ ]:
display(df.isna().sum())

price                   0
vehicletype         37490
registrationyear        0
gearbox             19833
power                   0
model               19705
kilometer               0
fueltype            32895
brand                   0
repaired            71154
dtype: int64

In [ ]:
print('Доля пропусков в стобце "vehicletype"', df['vehicletype'].isna().sum() / df.shape[0])
print('Доля пропусков в стобце "gearbox"', df['gearbox'].isna().sum() / df.shape[0])
print('Доля пропусков в стобце "model"', df['model'].isna().sum() / df.shape[0])
print('Доля пропусков в стобце "fueltype"', df['fueltype'].isna().sum() / df.shape[0])
print('Доля пропусков в стобце "repaired"', df['repaired'].isna().sum() / df.shape[0])

Доля пропусков в стобце "vehicletype" 0.1057936783409384
Доля пропусков в стобце "gearbox" 0.055967085156997366
Доля пропусков в стобце "model" 0.055605879746817584
Доля пропусков в стобце "fueltype" 0.09282696849893755
Доля пропусков в стобце "repaired" 0.20079070121822168


In [ ]:
df.isna().sum() / df.shape[0]

price               0.000000
vehicletype         0.105794
registrationyear    0.000000
gearbox             0.055967
power               0.000000
model               0.055606
kilometer           0.000000
fueltype            0.092827
brand               0.000000
repaired            0.200791
dtype: float64

In [ ]:
df_new = df.dropna()
display(df_new.isna().sum())

price               0
vehicletype         0
registrationyear    0
gearbox             0
power               0
model               0
kilometer           0
fueltype            0
brand               0
repaired            0
dtype: int64

In [ ]:
display(df_new.shape[0] / df.shape[0])

0.6936667710776055

Слишком большое количесто пропусков относительно исходной ткаблицы, также наш таргетный столбец 'price' без пропусков. Удалить их мы не можем, но так как неоткуда взять данные, заполним значением 'unknown'.

In [ ]:
df = df.fillna('unknown')
df.isna().sum()

price               0
vehicletype         0
registrationyear    0
gearbox             0
power               0
model               0
kilometer           0
fueltype            0
brand               0
repaired            0
dtype: int64

### Обработка дубликатов

In [ ]:
df.duplicated().sum() / df.shape[0]

0.1270991537070116

Думаю, что дубликаты возникли в связи с ошибкой при внесении данных. Удалю их.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

0

### Обработка аномалий

In [ ]:
df.describe()

,price,registrationyear,power,kilometer
count,309329.000000,309329.000000,309329.000000,309329.000000
mean,4486.937196,2004.360105,110.976908,127217.735809
std,4564.852796,92.541399,200.969473,38532.941010
min,0.000000,1000.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,69.000000,125000.000000
50%,2800.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Замечу, что цена не может быть нулём, поэтому возьму за минимальное значение значение первого квантиля (только если кто-то отдаёт машину бесплатно). Год регистрации также не может быть значением 1000, возьму за минимальное значение также значение первого квантиля, максимальне значение в виде 9999 также невозможно, поэтому огриничимся значением 2023. Мощность также не может быть 0, поэтому также возьму значение первого квантиля за минимальное, максимальное возьму 2000, так как это максимальная мощность на сегодняшний день.

In [ ]:
df = df.loc[df['price'] >= 1100]
df = df.loc[(df['registrationyear'] >= 1999) & (df['registrationyear'] <= 2023)]
df = df.loc[((df['power'] >= 69) & (df['power'] <= 2000))]

In [ ]:
df.describe()

,price,registrationyear,power,kilometer
count,167967.00000,167967.000000,167967.000000,167967.000000
mean,6682.74776,2006.247805,137.955426,124349.782993
std,4703.53844,4.589922,62.173926,38678.834600
min,1100.00000,1999.000000,69.000000,5000.000000
25%,2900.00000,2003.000000,101.000000,100000.000000
50%,5350.00000,2006.000000,129.000000,150000.000000
75%,9390.00000,2009.000000,163.000000,150000.000000
max,20000.00000,2019.000000,2000.000000,150000.000000


### Небольшой вывод по первому пункту

Изучил данные, удалил ненужные столбцы, обработал пропуски и дубликаты. Также обработал аномалии.

## Обучение моделей

In [ ]:
STATE = 2802

Разделим на выборки.

In [ ]:
features = df.drop('price', axis=1)
target = df['price']
train_features, valid_test_features, train_target, valid_test_target = train_test_split(features, target, test_size=0.4, random_state=STATE)
valid_features, test_features, valid_target, test_target = train_test_split(valid_test_features, valid_test_target, test_size=0.5, random_state=STATE)

display(train_features.shape)
display(train_target.shape)
display(valid_features.shape)
display(valid_target.shape)
display(test_features.shape)
display(test_target.shape)

(100780, 9)

(100780,)

(33593, 9)

(33593,)

(33594, 9)

(33594,)

### Стандартизация данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167967 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   price             167967 non-null  int64 
 1   vehicletype       167967 non-null  object
 2   registrationyear  167967 non-null  int64 
 3   gearbox           167967 non-null  object
 4   power             167967 non-null  int64 
 5   model             167967 non-null  object
 6   kilometer         167967 non-null  int64 
 7   fueltype          167967 non-null  object
 8   brand             167967 non-null  object
 9   repaired          167967 non-null  object
dtypes: int64(4), object(6)
memory usage: 14.1+ MB


In [ ]:
numeric = ['registrationyear', 'power', 'kilometer']

scaler = StandardScaler()
scaler.fit(train_features[numeric])
train_features[numeric] = scaler.transform(train_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])
test_features[numeric] = scaler.transform(test_features[numeric])

/tmp/ipykernel_31/2921458825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features[numeric] = scaler.transform(train_features[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_31/2921458825.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

### CatBoost

In [ ]:
%%time

cat_features = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'repaired']

model_cat_boost = CatBoostRegressor(loss_function='RMSE', depth=5, iterations=400)
model_cat_boost.fit(train_features, train_target, cat_features=cat_features, verbose=10)
predictions = model_cat_boost.predict(valid_features)
print('RMSE', sqrt(mean_squared_error(valid_target, predictions)))

Learning rate set to 0.178747
0:	learn: 4252.3123584	total: 169ms	remaining: 1m 7s
10:	learn: 2407.1764120	total: 1.26s	remaining: 44.5s
20:	learn: 2094.1139841	total: 2.03s	remaining: 36.7s
30:	learn: 1995.4060636	total: 2.83s	remaining: 33.6s
40:	learn: 1943.3644455	total: 3.63s	remaining: 31.8s
50:	learn: 1909.6893853	total: 4.4s	remaining: 30.1s
60:	learn: 1882.7136172	total: 5.09s	remaining: 28.3s
70:	learn: 1861.6590355	total: 5.94s	remaining: 27.5s
80:	learn: 1845.3731802	total: 6.79s	remaining: 26.8s
90:	learn: 1830.6128348	total: 7.51s	remaining: 25.5s
100:	learn: 1817.9263496	total: 8.23s	remaining: 24.4s
110:	learn: 1806.7353701	total: 8.97s	remaining: 23.3s
120:	learn: 1798.7795019	total: 9.75s	remaining: 22.5s
130:	learn: 1790.9232579	total: 10.4s	remaining: 21.4s
140:	learn: 1784.1754896	total: 11.2s	remaining: 20.7s
150:	learn: 1776.7906169	total: 12.1s	remaining: 19.9s
160:	learn: 1773.5170610	total: 12.8s	remaining: 19s
170:	learn: 1768.0665895	total: 13.5s	remaining: 

### Прямое кодирование

In [ ]:
df_ohe = pd.get_dummies(df, drop_first=True)
display(df.shape)
display(df_ohe.shape)

(167967, 10)

(167967, 302)

In [ ]:
features_ohe = df_ohe.drop('price', axis=1)
target_ohe = df_ohe['price']
train_features_ohe, valid_test_features_ohe, train_target_ohe, valid_test_target_ohe = train_test_split(features_ohe, target_ohe, test_size=0.4, random_state=STATE)
valid_features_ohe, test_features_ohe, valid_target_ohe, test_target_ohe = train_test_split(valid_test_features_ohe, valid_test_target_ohe, test_size=0.5, random_state=STATE)

display(train_features_ohe.shape)
display(train_target_ohe.shape)
display(valid_features_ohe.shape)
display(valid_target_ohe.shape)
display(test_features_ohe.shape)
display(test_target_ohe.shape)

(100780, 301)

(100780,)

(33593, 301)

(33593,)

(33594, 301)

(33594,)

### LightGBM

In [ ]:
%%time

best_rmse = 1000000
n_estimators = 0
max_depth = 0
max_leaves = 0

for n_estimators in tqdm(range(1,11, 2)):
    for max_depth in range(1, 5, 2):
        for max_leaves in range(1, 5, 2):
            model_lgbm = LGBMRegressor(n_estimators=n_estimators, max_depth=max_depth, max_leaves=max_leaves)
            model_lgbm.fit(train_features_ohe, train_target_ohe)
            predictions = model_lgbm.predict(valid_features_ohe)
            rmse = sqrt(mean_squared_error(valid_target_ohe, predictions))
    if rmse < best_rmse:
        best_rmse = rmse
        best_n_estimators = n_estimators
        best_max_depth = max_depth
        best_max_leaves = max_leaves

print('Лучшее количество деревьев:', best_n_estimators)
print('Лучшая глубина деревьев:', best_max_depth)
print('Лучшее количество терминальных узлов:', best_max_leaves)
print('Лучший квадратный корень среднеквадратичной ошибки:', best_rmse)

  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31


 20%|██        | 1/5 [00:11<00:46, 11.73s/it]

[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31


 40%|████      | 2/5 [00:27<00:42, 14.30s/it]

...

[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31


 80%|████████  | 4/5 [01:12<00:19, 19.79s/it]

[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3 will be ignored. Current value: num_leaves=31


100%|██████████| 5/5 [01:37<00:00, 19.56s/it]

Лучшее количество деревьев: 9
Лучшая глубина деревьев: 3
Лучшее количество терминальных узлов: 3
Лучший квадратный корень среднеквадратичной ошибки: 3219.24422372322
CPU times: user 1min 32s, sys: 4.43 s, total: 1min 37s
Wall time: 1min 37s


### RandomForestRegressor

In [ ]:
%%time

best_rmse = 1000000
n_estimators = 0
max_depth = 0

for n_estimators in tqdm(range(1,11, 2)):
    for max_depth in range(1, 5, 2):
        model_rfr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
        model_rfr.fit(train_features_ohe, train_target_ohe)
        predictions = model_rfr.predict(valid_features_ohe)
        rmse = sqrt(mean_squared_error(valid_target_ohe, predictions))
    if rmse < best_rmse:
        best_rmse = rmse
        best_n_estimators = n_estimators
        best_max_depth = max_depth

print('Лучшее количество деревьев:', best_n_estimators)
print('Лучшая глубина деревьев:', best_max_depth)
print('Лучший квадратный корень среднеквадратичной ошибки:', best_rmse)

100%|██████████| 5/5 [00:14<00:00,  2.99s/it]

Лучшее количество деревьев: 5
Лучшая глубина деревьев: 3
Лучший квадратный корень среднеквадратичной ошибки: 3110.0914000801968
CPU times: user 13.5 s, sys: 1.45 s, total: 14.9 s
Wall time: 14.9 s


Выберем CatBoost, так как среднеквадратичная ошибка ниже на порядок, а время по сравнению с случайным лесом, который работает 10 секунд, самой быстрой моделью из представленных, отличается на 20 секунд. Считаю это некритичным, так как ошибка отличается почти в два раза.

## Анализ моделей

In [ ]:
%%time

predictions = model_cat_boost.predict(test_features)
print('RMSE', sqrt(mean_squared_error(test_target, predictions)))

RMSE 1738.2963981695368
CPU times: user 105 ms, sys: 7.76 ms, total: 112 ms
Wall time: 109 ms


Получил отличный результат за достаточное короткое время выполнения кода.

## Общий вывод

Я изучил данные, удалил ненужные столбцы, обработал пропуски и дубликаты. Также мною были обработаны аномалии. Разделил на выборки, провёл стандартизацию данных. При помощи градиентного бустинга CatBoost и LightGBM, а также RandomForestRegressor получил среднеквадричтную ошибку. Для LightGBM и RandomForestRegressor произвёл прямое кодирование.

По данным выяснил, что лучше всего результат у CatBoost, проверил его на тестовых данных и получил маленькое значение среднеквадратичной ошибки за крайне короткое время выполнение кода.